In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import string
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
warnings.filterwarnings('ignore')

# Objetivo: Entender o perfil dos clientes do estado de SP

In [2]:
df_customers = pd.read_csv("data/olist_customers_dataset.csv")
df_orders = pd.read_csv("data/olist_orders_dataset.csv")
df_oitens = pd.read_csv("data/olist_order_items_dataset.csv")
df_prodts = pd.read_csv("data/olist_products_dataset.csv")
df_pays = pd.read_csv("data/olist_order_payments_dataset.csv")
df_geo = pd.read_csv("data/olist_geolocation_dataset.csv")

In [3]:
# Filtrando clientes de SP
customers_sp = df_customers[df_customers['customer_state'] == "SP"]

In [4]:
customers_sp.shape

(41746, 5)

In [5]:
raw_data = customers_sp.join(df_orders.set_index('customer_id'), on='customer_id', how = 'left').join(
    df_pays.set_index('order_id'), on = 'order_id', how = 'left').join(df_oitens.set_index('order_id'), 
                                                                       on = 'order_id', how = 'left')

In [6]:
raw_data.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,...,1.0,credit_card,2.0,146.87,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,...,1.0,credit_card,8.0,335.48,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,...,1.0,credit_card,7.0,157.73,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,...,1.0,credit_card,1.0,173.30,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,...,1.0,credit_card,8.0,252.25,1.0,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


In [7]:
raw_data.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

## Q1. Qual a proporção de clientes recorrentes? 

In [8]:
# Agrupando por clientes e contando quantas ordens de compra ele emitiu.
df_oders_sp = customers_sp.loc[:, ['customer_id']].join(df_orders.set_index('customer_id'), on='customer_id', how='left')
clies_recorrentes = df_oders_sp.groupby('customer_id')['order_id'].count().reset_index().sort_values(
    'order_id', ascending=False).rename(columns={'order_id':'qtd_recorrencia'})

In [9]:
clies_recorrentes.head()

,customer_id,qtd_recorrencia
0,00012a2ce6f8dcda20d059ce98491703,1
27844,aa1922a8ba370a11a86c1ad869c9f370,1
27826,aa04f4a6ef53a5e85ea9e6fb2a89d043,1
27827,aa0533eb31ed3be79086f11bb2bec430,1
27828,aa072112a00df3b18830253b7fd1f47f,1


In [10]:
clies_recorrentes.qtd_recorrencia.unique()

array([1], dtype=int64)

Normalmente, um cliente pode emitir mais de uma ordem compra, neste caso cada cliente só tem uma ordem de compra.
Os proprietários dos dados acabaram filtrando clientes que só tinham uma ordem de compra.

## Q2. Quais as cidades com maior número de clientes?

In [11]:
geo_sp = df_geo[df_geo['geolocation_state'] == 'SP']

In [12]:
letras_pontuadas = ['ã', 'â', 'ó', 'ú', 'é', 'ç', 'í', 'ü', 'ô']
letras = ['a', 'a', 'o', 'u', 'e', 'c', 'i', 'u', 'o']

In [13]:
cidades = ['Birigüi', 'São Paulo', 'Santa-Catarina', "Del' Vale"]

In [14]:
def remove(cidades):
    cidades_novas = []
    for cidade_index in range(len(cidades)):
        print(cidades[cidade_index])
        for i in range(len(cidades[cidade_index])):
            for p, l in zip(letras_pontuadas, letras):
                if cidades[cidade_index][i] == p:
                    cidades[cidade_index] = cidades[cidade_index].replace(cidades[cidade_index][i], l)
                if cidades[cidade_index][i] in string.punctuation:
                    cidades[cidade_index] = cidades[cidade_index].replace(cidades[cidade_index][i], " ")
        cidades_novas.append(cidades[cidade_index])
    return cidades_novas

In [15]:
geo_sp

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
404265,19970,-22.779062,-50.218287,palmital,SP
404266,19905,-22.997601,-49.885928,ourinhos,SP
404267,19907,-22.978498,-49.875919,ourinhos,SP
404268,19905,-22.993127,-49.887665,ourinhos,SP


In [63]:
df_cons = df_customers[df_customers['customer_state'] == 'SP'].join(df_orders.set_index('customer_id'), on='customer_id', how='left').join(df_pays.set_index('order_id'), on='order_id', how='left').join(df_oitens.set_index('order_id'), on='order_id', how='left').join(df_prodts.set_index('product_id'), on='product_id', how='left')

In [17]:
df_cons.shape

(49967, 30)

In [18]:
df_cons.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,...,124.99,21.88,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,...,289.00,46.48,utilidades_domesticas,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,...,139.94,17.79,moveis_escritorio,55.0,955.0,1.0,8267.0,52.0,52.0,17.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,...,149.94,23.36,moveis_escritorio,48.0,1066.0,1.0,12160.0,56.0,51.0,28.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,...,230.00,22.25,casa_conforto,61.0,407.0,1.0,5200.0,45.0,15.0,35.0


In [19]:
df_cons.customer_city.nunique()

629

In [64]:
cidades_publico = df_cons.groupby('customer_city')['customer_id'].count().reset_index().sort_values(by='customer_id', ascending=False)

In [21]:
locator = Nominatim(user_agent='MyGeocoder')

In [91]:
cidades_info = {}
cidades_info['nome_cidade'] = []
cidades_info['lat'] = []
cidades_info['long'] = []
for cidade in cidades_publico.customer_city.unique():
    cidade_buscada = locator.geocode('{}, são paulo, Brasil'.format(cidade))
    cidades_info['nome_cidade'].append(cidade)
    lat, long = cidade_buscada[1]
    cidades_info['lat'].append(lat)
    cidades_info['long'].append(long)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=santa+barbara+d+oeste%2C+s%C3%A3o+paulo%2C+Brasil&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000189C873A160>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

In [ ]:
lat_long = pd.DataFrame.from_dict(cidades_info)
lat_long.to_csv('lat_long.csv', index=False)

In [ ]:
cidades_publico['lat'] = cidades_info['lat']
cidades_publico['long'] = cidades_info['long']

In [ ]:
cidades_publico

In [ ]:
bbox = ((cidades_publico.lat.min(), cidades_publico.lat.max()), 
        (cidades_publico.long.min(), cidades_publico.long.max()))

In [ ]:
from plotly.express import scatter_geo

In [ ]:
cidades_publico[cidades_publico['long'] == -53.2]

In [ ]:
scatter_geo(cidades_publico, lat='lat', lon='long', labels='customer_city')